In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import math
import sympy as sp
import random
from tqdm import tqdm

In [2]:
df = pd.read_csv('Salary_dataset.csv')
df.drop(columns='Unnamed: 0',axis=1, inplace=True)

In [3]:
def min_max_normalize(column):
    return (column - column.min()) / (column.max() - column.min())
columns_to_normalize = df.columns

df[columns_to_normalize] = df[columns_to_normalize].apply(min_max_normalize)

In [4]:
def previsao(df,w,b):
    lista_predicto = []
    lista_real = []
    linhas, colunas  = df.shape
    for linha in range(linhas):
        total = 0
        for coluna in range(colunas-1):
            x = df.iloc[linha,coluna]
            total += x * w[coluna]
        y_predicto = total + b
        lista_predicto.append(y_predicto)
    lista_real = df.iloc[:,colunas-1].tolist()
    return lista_predicto, lista_real

In [5]:
def n_interacao(interacao,n):
    return n/interacao

<b>Função linear: </b><br>
$f(x)=x$

<b>Calculando o gradiente descendente para o peso </b> <br><br>

$\frac{\partial E}{\partial \omega_k} = 
\frac{1}{N} \sum_{i=1}^N \frac{\partial (\hat{y}-y)^2}{\partial \omega_k}$ <br><br>

$=\frac{1}{N} \sum_{i=1}^N \frac{\partial (\hat{y}_i-y_i)^2}{\partial (\hat{y}_i-y_i)}$ 
$\cdot\frac{\partial (\hat{y}_i-y_i)}{\partial \hat{y}_i}$
$\cdot \frac{\partial \hat{y}_i}{\partial \omega_{k}}$ <br><br> 

$=\frac{1}{N} \sum_{i=1}^N \frac{\partial (\hat{y}-y)^2}{\partial (\hat{y}-y)}$ 
$\cdot\frac{\partial (\hat{y}-y)}{\partial \hat{y}}$
$\cdot \frac{\partial f_a(sum)}{\partial sum}$
$\cdot \frac{\partial sum}{\partial \omega_{k}}$ <br><br> 

Considerando a função de ativação linear: <br><br>
$=\frac{1}{N} \sum_{i=1}^N 2\cdot(\hat{y}_i-y_i) \cdot 1 \cdot 1 \cdot x_i^k$ <br><br>

<b>Fórmula final para atualizar o peso</b> <br><br>
Considerando a função de ativação linear: <br><br>

$\omega_{k}^{t} = \omega_{k}^{t-1} - \eta^{t} \cdot\frac{1}{N} \sum_{i=1}^N 2\cdot(\hat{y}_i-y_i) \cdot 1 \cdot 1 \cdot x_i^k$ <br><br>

<b>Calculando o gradiente descendente para o bias </b> <br><br>

$\frac{\partial E}{\partial b_k} = 
\frac{1}{N} \sum_{i=1}^N \frac{\partial (\hat{y}-y)^2}{\partial b_k}$ <br><br>

$=\frac{1}{N} \sum_{i=1}^N \frac{\partial (\hat{y}_i-y_i)^2}{\partial (\hat{y}_i-y_i)}$ 
$\cdot\frac{\partial (\hat{y}_i-y_i)}{\partial \hat{y}_i}$
$\cdot \frac{\partial \hat{y}_i}{\partial b_{k}}$ <br><br> 

$=\frac{1}{N} \sum_{i=1}^N \frac{\partial (\hat{y}-y)^2}{\partial (\hat{y}-y)}$ 
$\cdot\frac{\partial (\hat{y}-y)}{\partial \hat{y}}$
$\cdot \frac{\partial f_a(sum)}{\partial sum}$
$\cdot \frac{\partial sum}{\partial b_{k}}$ <br><br> 

Considerando a função de ativação linear: <br><br>
$=\frac{1}{N} \sum_{i=1}^N 2\cdot(\hat{y}_i-y_i) \cdot 1 \cdot 1 \cdot 1$ <br><br>


<b>Fórmula final para atualizar o bias</b> <br><br>
Considerando a função de ativação linear: <br><br>

$b_{k}^{t} = b_{k}^{t-1} - \eta^{t} \cdot\frac{1}{N} \sum_{i=1}^N 2\cdot(\hat{y}_i-y_i) \cdot 1 \cdot 1 \cdot 1$ <br><br>

Obs: Como discutido em aula iremos desconsiderar o "2" em nossos cálculos

In [6]:
def atualizacao_peso(lista_predicto,lista_real,df,w,n,b,interacao):
    
    w_novo = []

    #n = n_interacao(interacao,n)

    linhas, colunas  = df.shape
    y_diferenca = [i1 - i2 for i1, i2 in zip(lista_predicto,lista_real)]
    for coluna in range(colunas-1):
        w_total = 0

        for linha in range(linhas):
            w_total += (y_diferenca[linha] * df.iloc[linha,coluna])
            
        w_total = w[coluna] - n * (w_total/linhas)
        w_novo.append(w_total)
        
    b_novo = sum(y_diferenca)
    b_novo /= linhas
    b_novo = b - n * b_novo

    return w_novo,b_novo

In [7]:
w = [random.gauss(0, 0.5) for _ in range(df.shape[1] - 1)]
b = random.gauss(0, 0.5)
n=0.4

In [8]:
interacao = 10
for i in tqdm(range(1,interacao)):
    if (interacao%11==0):
        n = n*0.9
    lista_predicto, lista_real = previsao(df,w,b)
    w, b = atualizacao_peso(lista_predicto,lista_real,df,w,n,b,i)

100%|████████████████████████████████████████████| 9/9 [00:00<00:00, 211.06it/s]


In [9]:
interacao = 100
for i in tqdm(range(1,interacao)):
    lista_predicto, lista_real = previsao(df,w,b)
    w, b = atualizacao_peso(lista_predicto,lista_real,df,w,n,b,i)

100%|██████████████████████████████████████████| 99/99 [00:00<00:00, 623.89it/s]


In [14]:
total = 0
for i in range(len(lista_predicto)):
    a = (df['Salary'].iloc[i] - lista_predicto[i])**2
    total += a
print(f'RMSE: {round((total/len(lista_predicto))**(1/2),4)}')
result = pd.DataFrame({'REAL':lista_real, 'PREDICTO':lista_predicto})
print(result.to_string(index=False))


RMSE: 0.0662
    REAL  PREDICTO
0.019041 -0.010086
0.100094  0.011879
0.000000  0.033845
0.068438  0.088759
0.025514  0.110725
0.223376  0.187605
0.264812  0.198588
0.197425  0.220553
0.315545  0.220553
0.229837  0.275468
0.301051  0.297433
0.213359  0.308416
0.227097  0.308416
0.228561  0.319399
0.276163  0.363330
0.356804  0.407262
0.334255  0.429227
0.535755  0.451193
0.515379  0.517090
0.663938  0.528073
0.637928  0.615935
0.715119  0.648884
0.750898  0.736747
0.898665  0.769695
0.846917  0.824609
0.801453  0.857558
0.935956  0.912472
0.884763  0.923455
1.000000  1.000335
0.993870  1.022300
